In [1]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt

# DESIGNEDDATAPATH = '../designed_data/cad/'

In [2]:
constructs = ['sexism', 'hatespeech']

In [3]:
construct = 'hatespeech'

In [4]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [5]:
paired_data = {}

train_data_path = '../data/data/%s/train/paired_cads_mixed.csv' %construct
paired_data[construct] = pd.read_csv(train_data_path, sep = '\t')
paired_data[construct]['polyjuice_id'] = [str(i)+'p' for i in paired_data[construct]['original_id']]
paired_data[construct]['chatgpt_id'] = [str(i)+'gpt' for i in paired_data[construct]['original_id']]
paired_data[construct]['flant5_id'] = [str(i)+'ft' for i in paired_data[construct]['original_id']]    

In [6]:
paired_data[construct].columns

Index(['Unnamed: 0', 'original_id', 'counterfactual_id', 'original_text',
       'counterfactual_text', 'original_label', 'counterfactual_label', 'diff',
       'negation_additions', 'negation_deletions', 'affect word_additions',
       'affect word_deletions', 'gender word_additions',
       'gender word_deletions', 'identity word_additions',
       'identity word_deletions', 'hedges_additions', 'hedges_deletions',
       'hate words_additions', 'hate words_deletions', 'polyjuice',
       'polyjuice_label', 'chatgpt', 'chatgpt_label', 'flant5', 'flant5_label',
       'polyjuice_text', 'chatgpt_text', 'flant5_text', 'mixed_cad_text',
       'mixed_cad_type', 'mixed_cad_label', 'polyjuice_id', 'chatgpt_id',
       'flant5_id'],
      dtype='object')

In [7]:
print(construct)
print(paired_data[construct].groupby('original_label').size())
print()
print(paired_data[construct].groupby('counterfactual_label').size())
print()
print(paired_data[construct].groupby('polyjuice_label').size())
print()
print(paired_data[construct].groupby('chatgpt_label').size())
print()
print(paired_data[construct].groupby('flant5_label').size())
print('---------------------------------------------------------')

hatespeech
original_label
hate        6524
not hate    5767
dtype: int64

counterfactual_label
hate        5780
not hate    6511
dtype: int64

polyjuice_label
MAR         5613
hate        3282
not hate    3396
dtype: int64

chatgpt_label
MAR          175
hate        5686
not hate    6430
dtype: int64

flant5_label
MAR         1069
hate        5136
not hate    6086
dtype: int64
---------------------------------------------------------


In [8]:
paired_data[construct].groupby('original_label').size()

original_label
hate        6524
not hate    5767
dtype: int64

In [9]:
paired_data[construct].groupby('chatgpt_label').size()

chatgpt_label
MAR          175
hate        5686
not hate    6430
dtype: int64

create a cvs file with the following:
- original_id (can repeat)
- text (unique)
- CAD source
- original_label
- (expected) CAD label

In [10]:
automated_cad_types = ['polyjuice', 'chatgpt', 'flant5']

In [11]:
annotatable = []
for cad_type in automated_cad_types:
    data = paired_data[construct].copy()
    data = data[~data[cad_type+'_label'].isin(['MAR', 'NMAR'])]
    data_sample = data.sample(100)
    data_sample = data_sample[['original_id', cad_type+'_text', 'original_label', cad_type+'_label']]
    data_sample.columns = ['original_id', 'CAD_text', 'original_label', 'expected_CAD_label']
    data_sample['CAD_source'] = [cad_type] * len(data_sample)
    annotatable.append(data_sample)
    print(cad_type, len(data), len(data_sample))
    
annotatable_df = pd.concat(annotatable)
# annotatable_df.columns = ['original_id', 'CAD_text', 'CAD_source', 'original_label', 'expected_CAD_label']
annotatable_df['ann_id'] = range(0, len(annotatable_df))
annotatable_df = annotatable_df.set_index('ann_id').sample(frac=1)
# annotatable_df.head(150).to_csv('../data/to_annotate/%s_CAD_label_batch1.csv' %construct, sep = '\t')
# annotatable_df.tail(150).to_csv('../data/to_annotate/%s_CAD_label_batch2.csv' %construct, sep = '\t')
annotatable_df

polyjuice 6678 100
chatgpt 12116 100
flant5 11222 100


,original_id,CAD_text,original_label,expected_CAD_label,CAD_source
ann_id,,,,,
162,32479,The fact that they must continually seek more ...,not hate,hate,chatgpt
271,46595,Fuck kikes and NEVER THINK FOR A SECOND THE SC...,hate,not hate,flant5
294,27566,I am seeing more hindu people in this area com...,not hate,hate,flant5
134,23248,This message contains hate speech and cannot b...,hate,not hate,chatgpt
152,20820,My fucking neighbour is a goddamn nuisance.,not hate,hate,chatgpt
...,...,...,...,...,...
241,32624,When Trevor Philips criticises your use of PC ...,not hate,hate,flant5
279,34307,"Usually the complaints are just "" why is there...",not hate,hate,flant5
173,31881,Racism is a problem that needs to be overcome....,hate,not hate,chatgpt


In [33]:
### analyze the annotations
annotated_df = {}
annotator = 'dennis' # 'indira' # 
annotated_df['sexism'] = pd.read_csv('../results/intermediate/manually_annotated/sexism_CAD_label_batch1 - %s.csv' %annotator)
annotated_df['sexism']

,ann_id,original_id,CAD_text,original_label,expected_CAD_label,CAD_source,sexist or not?,"if the text is grammatically incoherent, leave blank"
0,257,5d654927ce2c6bd733479744,I got used to the idea of seeing girls on skat...,sexist,non-sexist,flant5,sexist,NaN
1,125,5d654921ce2c6bd733478d1b,I dislike when I see one person in a relations...,sexist,non-sexist,chatgpt,non-sexist,NaN
2,32,5ddc4fe8174b03a4c6aaf7cd,Parents usually maintain stricter control over...,sexist,non-sexist,polyjuice,sexist,NaN
3,134,5d654921ce2c6bd733478cb8,I can't take female broadcasters on ESPN serio...,sexist,non-sexist,chatgpt,sexist,NaN
4,74,5d654923ce2c6bd7334791f0,ponytails should not be found on the head of a...,sexist,non-sexist,polyjuice,sexist,implicit
...,...,...,...,...,...,...,...,...
145,62,5d654922ce2c6bd733478f0a,the women on ESPN are great as fuck.,sexist,non-sexist,polyjuice,non-sexist,NaN
146,119,5d654922ce2c6bd733478f2d,I prefer sports announcers who are knowledgeab...,sexist,non-sexist,chatgpt,non-sexist,NaN
147,122,5d654922ce2c6bd73347907a,I never listen to anything the women or men sp...,sexist,non-sexist,chatgpt,non-sexist,NaN
148,244,5d654922ce2c6bd733478fc1,"women talking sports makes me happy #mansworld""",sexist,non-sexist,flant5,sexist,NaN


In [34]:
annotated_df['sexism'].groupby(['CAD_source', 'sexist or not?']).size().unstack() 

sexist or not?,non-sexist,sexist
CAD_source,,
chatgpt,42,2
flant5,12,37
polyjuice,11,46


In [35]:
annotated_df['sexism'].groupby(['sexist or not?']).size() / len(annotated_df['sexism'])

sexist or not?
non-sexist    0.433333
sexist        0.566667
dtype: float64

In [43]:
### analyze the annotations
annotated_df = {}
annotator = 'dennis' # 'indira' # 
annotated_df['hatespeech'] = pd.read_csv('../results/intermediate/manually_annotated/hatespeech_CAD_label_batch1 - %s.csv'
                                        %annotator)
annotated_df['hatespeech']

,ann_id,original_id,CAD_text,original_label,expected_CAD_label,CAD_source,hate or not?,Unnamed: 7,Unnamed: 8
0,290,31770,I knew many Russian/Ukranian girls who wish to...,hate,not hate,flant5,hate,NaN,NaN
1,94,20887,"There are many immigrants coming here, you wil...",not hate,hate,polyjuice,not hate,NaN,NaN
2,40,43876,The female equality movement is pretty much an...,hate,not hate,polyjuice,NaN,NaN,NaN
3,284,24632,this lotion should be allowed in this country,not hate,hate,flant5,not hate,NaN,NaN
4,185,26262,No modification can make this sentence non-hat...,hate,not hate,chatgpt,not hate,funny,error response
...,...,...,...,...,...,...,...,...,...
145,22,32799,I saw a clip of this absolute genius from luna...,hate,not hate,polyjuice,not hate,NaN,NaN
146,190,21724,I do not support the formation of neo-nazi pol...,hate,not hate,chatgpt,not hate,NaN,NaN
147,223,25035,these false eyelashes are one of the reasons w...,not hate,hate,flant5,not hate,NaN,NaN
148,178,25288,This is getting ridiculous. It's not right to ...,hate,not hate,chatgpt,not hate,NaN,NaN


In [44]:
annotated_df['hatespeech'].groupby(['CAD_source', 'expected_CAD_label', 'hate or not?']).size().unstack()

hate or not?                   hate  not hate
CAD_source expected_CAD_label                
chatgpt    hate                  15         8
           not hate               4        18
flant5     hate                   4        22
           not hate              19         1
polyjuice  hate                   3        22
           not hate              21        12

In [45]:
annotated_df['hatespeech'].groupby(['CAD_source', 'hate or not?']).size().unstack() 

hate or not?,hate,not hate
CAD_source,,
chatgpt,19,26
flant5,23,23
polyjuice,24,34


In [46]:
annotated_df['hatespeech']['mislabeled'] = ['mislabeled' if row['hate or not?'] != row['expected_CAD_label'] else 'correct' for n,
                                           row in annotated_df['hatespeech'].iterrows()]
annotated_df['hatespeech'].groupby('mislabeled').size() / len(annotated_df['hatespeech'])

mislabeled
correct       0.353333
mislabeled    0.646667
dtype: float64

In [47]:
chatgpt = annotated_df['hatespeech'][annotated_df['hatespeech']['CAD_source'] == 'chatgpt']
len(chatgpt[chatgpt['mislabeled'] == 'mislabeled']) / len(chatgpt)

0.26666666666666666

#### get agreement

In [50]:
from sklearn.metrics import cohen_kappa_score

In [71]:
for annotator in ['indira', 'dennis']:
    annotated_df[annotator] = pd.read_csv('../results/intermediate/manually_annotated/sexism_CAD_label_batch1 - %s.csv' %annotator)
labeler1= list(pd.factorize(annotated_df['indira']['sexist or not?'].values)[0])
labeler2 = list(pd.factorize(annotated_df['dennis']['sexist or not?'].values)[0])

In [73]:
cohen_kappa_score(labeler1, labeler2)

0.6718322698268004

In [74]:
for annotator in ['indira', 'dennis']:
    annotated_df[annotator] = pd.read_csv('../results/intermediate/manually_annotated/hatespeech_CAD_label_batch1 - %s.csv' %annotator)
labeler1= list(pd.factorize(annotated_df['indira']['hate or not?'].values)[0])
labeler2 = list(pd.factorize(annotated_df['dennis']['hate or not?'].values)[0])
cohen_kappa_score(labeler1, labeler2)

0.7851772287862513